# Generating AI writing - Example for setting up feedback task with Log10

## Install dependencies

In [1]:
!pip install openai
!pip install -U log10-io
!pip install magentic

  Using cached magentic-0.21.1-py3-none-any.whl (39 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
  Using cached pydantic_settings-2.2.1-py3-none-any.whl (13 kB)


### Set up env vars

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "" #replace with your key

os.environ["LOG10_URL"]="https://log10.io"
os.environ["LOG10_ORG_ID"]=""  #replace with your credentials from Log10
os.environ["LOG10_TOKEN"]=""  #replace with your token from Log10

In [3]:
import asyncio
import openai
from magentic import AsyncStreamedStr, prompt
import uuid

from log10.load import log10, log10_session
log10(openai)

# Tag selector below is used for:
# 1) Selecting which logs to show in the Feedback inbox in the GUI
# 2) Tagging new logs/completions to attach feedback to
task_tag_selector = "twain-SDK-test4"

## Create task

In [4]:
from log10.feedback.feedback import Feedback
from log10.feedback.feedback_task import FeedbackTask

task = FeedbackTask().create(
    name="TASK_NAME:" + task_tag_selector,
    task_schema={
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "How well does the generated text satisfy user's requirements",
    "description": "Schema for positive, neutral or negative feedback",
    "type": "object",
    "properties": {
      "CHANNEL Email": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the Email channel?"
      },
      "CHANNEL LinkedIn": {
          "type": "string",
          "enum": [
            "😀",
            "😐",
            "😫",
            "N/A"
          ],
          "description": "How well does generation adhere to the LinkedIn channel?"
    },
      "USE CASE Connect request": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the connect request use case?"
    },
    "USE CASE Introduction": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the introduction use case?"
    },
    "USE CASE Reply": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the reply use case?"
    },
    "USE CASE Follow up": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the follow up use case?"
    },
    "GOAL Check relevancy": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the goal of check relevancy?"
    },
    "GOAL Book a call": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the goal of booking a call?"
    },
    "GOAL Create Interest": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the goal of create interest?"
    },
    "TONE Informal": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the informal tone?"
    },
    "TONE Direct": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the direct tone?"
    },
    "TONE Creative": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the creative tone?"
    },
    "TONE Bold": {
        "type": "string",
        "enum": [
          "😀",
          "😐",
          "😫",
          "N/A"
        ],
        "description": "How well does generation adhere to the bold tone?"
    }
  }
},
    completion_tags_selector=[task_tag_selector], # 1) Selecting which logs to show in the Feedback inbox in the GUI
    instruction="Please specify how well the generated text satisfies the relevant user selected criteria?",
)

### Test prompt for generative writing use case

In [5]:
@prompt("Create a outbound sales message for {channel} for {use_case} with the goal of {goal}. \
The tone should be {tone}. The company name is {company_name}. \
The problem they are tacking is {problem}. \
Their solution is {solution}. \
The results you can expect are {results}.\
Remember LinkedIn connection requests need to be 300 chars or shorter.")
async def create_message(channel: str, use_case: str, goal: str, tone: str,
                         company_name: str, problem: str, solution: str,
                         results: str) -> AsyncStreamedStr:  # ruff: ignore
  ...


## Generate Example 1

In [6]:
uuid_str = str(uuid.uuid4())

async def main():
  # Example 1
  channel = "LinkedIn"
  use_case = "Connection request"
  goal = "Book a call"
  tone = "Direct"
  company_name = "Log10"
  problem = "Currently most LLM apps are inaccurate limiting user adoption"
  solution = "We help you measure and improve accuracy of LLM apps"
  results = "Improve accuracy by 10-20 F1 points"


  with log10_session(tags=[task_tag_selector, channel, use_case, goal, tone, uuid_str,
                           "problem: "+ problem, "solution: "+solution,
                           "results: "+results]): # assigning metadata as tags can be useful in having relevant info on hand to help with the review in the UI
      output = await create_message(channel, use_case, goal, tone, company_name,
                                    problem, solution, results)
      async for chunk in output:
          print(chunk, end="", flush=True)


await main()

Subject: Boost Your LLM App’s Accuracy

Hi [Name], 

I'm with Log10, where we enhance LLM apps’ accuracy by 10-20 F1 points, overcoming common adoption barriers due to inaccuracies. Let’s discuss how we can elevate your app’s performance. Can we book a call this week?

Best, [Your Name]

### Assign Feedback on Example 1 via SDK/API

In [7]:
task_id = task.json()["id"]

Feedback().create(
    task_id=task_id,
    values={
        "TONE Bold":"😀",
        "TONE Creative":"😫",
        },
    completion_tags_selector=[uuid_str, task_tag_selector], # 2) Tagging new logs/completions to attach feedback to
)

<Response [200 OK]>

## Generate Example 2

In [8]:
uuid_str = str(uuid.uuid4())

async def main():
  # Example 2
  channel = "Email"
  use_case = "Introduction"
  goal = "Create interest"
  tone = "Creative"
  company_name = "Log10"
  problem = "Currently most LLM apps are inaccurate limiting user adoption"
  solution = "We help you measure and improve accuracy of LLM apps"
  results = "Improve accuracy by 10-20 F1 points"


  with log10_session(tags=[task_tag_selector, channel, use_case, goal, tone, uuid_str,
                           "problem: "+ problem, "solution: "+solution,
                           "results: "+results]):
      output = await create_message(channel, use_case, goal, tone, company_name,
                                    problem, solution, results)
      async for chunk in output:
          print(chunk, end="", flush=True)


await main()

Subject: 🚀 Elevate Your LLM Apps with Precision—Let's Chat!

Hi [Recipient’s Name],

I'm reaching out from Log10, where we've cracked the code on enhancing LLM app accuracy—a challenge I notice many face in the tech sphere!

Have you ever imagined boosting your app’s accuracy by 10-20 F1 points? That's exactly what we offer. Let’s explore how Log10 can turn this into reality for your projects, ensuring better adoption and user satisfaction.

Keen to learn more? Let’s schedule a quick call to dive deeper!

Best,
[Your Name]

### Assign Feedback via the UI
1. Log in to your log10.io account
2. Navigate to the Feedback tab in the left side panel
3. You should see the above example in the inbox. Click on it to assign feedback via the UI

## Talk to us / Coming soon
1. Use Log10's AutoFeedback to automatically assign feedback to new generations from the LLM
2. Get summary stats over feedback (human + AutoFeedback)
3. Set up alerts over AutoFeedback
4. Curate datasets for fine-tuning or prompt engineering to improve accuracy and customize per customer